In [1]:
import json
import pandas as pd
import re

def remove_dupe_rules(rules):
    list_of_strings = [json.dumps(d, sort_keys=True) for d in rules]
    list_of_strings = set(list_of_strings)
    return [json.loads(s)for s in list_of_strings]

In [2]:
import csv
def defineChannels(obj, event):
    channelFile = open('Channels.csv')
    channels = csv.reader(channelFile)
    for row in channels:
        if row[0] in obj and row[1] in event:
            return [row[2], int(row[3])]
    return -1
        
channel = defineChannels("switch", "<=")

In [3]:
file = 'information_malicious_apps.json'
with open(file) as ta_file:
    trigger_action = json.load(ta_file)
file_name = list(trigger_action.keys())
trigger_action_rules = dict()
ruleCount = 0
for app in file_name:
    rules = list()
    list_action = trigger_action[app]['list_action']
    list_sink = trigger_action[app]['list_sink']
    capabilities = trigger_action[app]['map_capa_inputs']
    
    for la in list_action:
        triggerAction = dict()
        action = dict()
        capability = dict()
        
        if la['action_cmd'] == None:
            continue
        triggerAction['trigger_device'] = la['trigger_var']
        triggerAction['trigger_attr'] = la['trigger_attr']
        triggerAction['action_device'] = la['action_var']
        triggerAction['action_cmd'] = la['action_cmd']
#         action['action_unit'] = app

        attr = str(triggerAction['trigger_attr'].split('.')[0])
        for cond in la['str_cond_path']:
            for condition in cond:
                foundCondition = False
                condition = condition.replace('(','')
                condition = condition.replace(')','')
                token = condition.split()
                for i in range(len(token)):
                    if token[i].startswith('evt'):
                        foundCondition = True
                        if token[i+1] == '==' or token[i+1] == 'in':
                            if token[i+2] == 'not':
                                attr = attr + '.' + token[i+2] + ' ' + token[i+3]
                            else:
                                attr = attr + '.' + token[i+2]
                        else:
                            attr = attr + ' ' + token[i+1] + ' ' + token[i+2]
                        triggerAction['trigger_attr'] = attr
                if foundCondition:
                    break
        action_ch = defineChannels(triggerAction['action_device'], triggerAction['action_cmd'])
        
        attr_sp_1 = triggerAction['trigger_attr'].split('.')
        attr_sp_2 = triggerAction['trigger_attr'].split(' ')
        if len(attr_sp_1) == 2:
            trigger_ch = defineChannels(attr_sp_1[0], attr_sp_1[1])
        elif len(attr_sp_2) == 3:
            trigger_ch = defineChannels(attr_sp_2[0], attr_sp_2[1])
        else: trigger_ch = -1
        
        triggerAction['action_channel'] = action_ch
        triggerAction['trigger_channel'] = trigger_ch
        
        rules.append(triggerAction)
        
    for ls in list_sink:
        
        action_cmd = ls['action_cmd']
        pattern = [r'.*send.*', r'.*http.*']
        if re.search(pattern[0],str(action_cmd),re.IGNORECASE) or re.search(pattern[1],str(action_cmd),re.IGNORECASE):
            triggerAction = dict()
            action = dict()
            capability = dict()

            triggerAction['trigger_device'] = ls['trigger_var']
            triggerAction['trigger_attr'] = ls['trigger_attr']
            triggerAction['action_device'] = app
            if re.search(r'.*sms.*',str(action_cmd),re.IGNORECASE):
                triggerAction['action_cmd'] = 'sending_sms'
            elif re.search(r'.*http.*',str(action_cmd),re.IGNORECASE):
                triggerAction['action_cmd'] = 'http_request'
            else:
                triggerAction['action_cmd'] = 'sending_notification'
                
            triggerAction['action_channel'] = -1
            triggerAction['trigger_channel'] = -1
            
#             attr = str(triggerAction['trigger_attr'].split('.')[0])
#             for cond in la['str_cond_path']:
#                 for condition in cond:
#                     foundCondition = False
#                     condition = condition.replace('(','')
#                     condition = condition.replace(')','')
#                     token = condition.split()
#                     for i in range(len(token)):
#                         if token[i].startswith('evt'):
#                             foundCondition = True
#                             if token[i+1] == '==' or token[i+1] == 'in':
#                                 if token[i+2] == 'not':
#                                     attr = attr + '.' + token[i+2] + ' ' + token[i+3]
#                                 else:
#                                     attr = attr + '.' + token[i+2]
#                             else:
#                                 attr = attr + ' ' + token[i+1] + ' ' + token[i+2]
#                             triggerAction['trigger_attr'] = attr
#                     if foundCondition:
#                         break
            
            rules.append(triggerAction)

    trigger_action_rules.update({app:remove_dupe_rules(rules)})

for app in trigger_action_rules:
    for rules in trigger_action_rules[app]:
        ruleCount += 1
        rules['ruleID'] = 'rule' +str(ruleCount)

In [4]:
with open('triggeraction_malicious.json','w') as f:
    json.dump(trigger_action_rules,f,indent=2)

In [ ]:
actions = []
ruleCount = 0
for app in trigger_action_rules:
    for rules in trigger_action_rules[app]:
        ruleCount += 1
        rules['ruleID'] = 'rule' +str(ruleCount)
        print(rules)
        actions.append(rules['action_cmd'])
action = remove_dupe_rules(actions)
action.sort()
print(action)